http://neurotycho.org/expdatalist/listview?task=78の
20110524KTMD_Anesthesia+and+Sleep_Kin2_Toru+Yanagawa_mat_ECoG128
を
$share/ECoGなどと名付けたフォルダ内にダウンロードする。

In [ ]:
#ダウンロードしたファイルをtrainとdevとtestに分ける
import scipy.io
import csv
import pandas as pd
import numpy as np
import scipy.io
import os
import random
import pandas as pd
import torch
import torchaudio
import csv
dataroot='$share/ECoG/20110524KTMD_Anesthesia+and+Sleep_Kin2_Toru+Yanagawa_mat_ECoG128/Session1/'
root='$share/ECoG_training/Kin2_20110524_session1_CommonAve_ReginLabel/'
test_list=[ 24, 62, 3, 49,11,  34, 54, 25, 53, 42, 50, 10, 8]
train_list=[7, 0, 45, 29, 36, 38, 66, 60, 40, 13, 2, 4, 39, 41, 51, 18, 44, 1, 52,  33, 17, 46, 26, 19,  61, 55, 16, 57, 63, 23, 14, 5, 67,  27, 65, 32, 48, 21, 31, 64, 30, 20, 56, 9, 22,15, 28, 47, 43, 37, 58, 35, 59, 6, 12]

region=[
    [1,2,3,4,9,10,17,18,26,27,36,37,47,48,56,57],#LP
    [5,6,7,8,11,12,13,14,19,20,21,22,28,29,38,39,49,50,65,66,74,75],#PM
    [15,16,23,24,25,30,31,32,33,40,41,42,43,51,52,58,59,60,61,67,68,76,77],#MS
    [34,35,44,45,53,54,55,64,69,70,71,72],#PC
    [78,79,95,96,97,98,107,108,109,110,111,112,113,122,123,124,125,126,127,128],#TC
    [73,83,84,80,81,82,99,100,101,102,114,115,116,117],#HV
    [46,85,86,87,88,89,90,91,92,93,94,103,104,105,106,118,119,120,121]
]
electrodes=[region[i][j] for i in range(len(region)) for j in range(len(region[i])) ]
num_electrodes=len(electrodes)
#print(electrodes,num_electrodes)
path=dataroot+'ECoG_ch1'+'.mat'
data_ECoG_ch= scipy.io.loadmat(path)
data=data_ECoG_ch['ECoGData_ch1'][0]
len_data=len(data)
lists_ECoG_CommonAve = np.array([0 for i in range(len_data)])
#num_CommonAve=np.array([i+1 for i in range(num_electrodes)])

for k in electrodes:
    num=k
    path=dataroot+'ECoG_ch'+str(num)+'.mat'
    data_ECoG_ch= scipy.io.loadmat(path)
    arg='ECoGData_ch'+str(num)    
    data=data_ECoG_ch[arg][0]
    print('data'+str(k),data)
    data=np.array(data[:2800000])
    lists_ECoG_CommonAve +=data
    #print('lists_ECoG_CommonAve'+str(k),lists_ECoG_CommonAve)
lists_ECoG_CommonAve=lists_ECoG_CommonAve/num_electrodes
#print('lists_ECoG_CommonAve',lists_ECoG_CommonAve)

for k in electrodes:
    num=k
    path=dataroot+'ECoG_ch'+str(num)+'.mat'
    data_ECoG_ch= scipy.io.loadmat(path)
    arg='ECoGData_ch'+str(num)    
    data=data_ECoG_ch[arg][0]
    data=np.array(data[:2800000])
    data=data-lists_ECoG_CommonAve
    #print('lists_ECoG_CommonAve',lists_ECoG_CommonAve)
    #print('data'+str(k),data)
    for j in train_list:
        for i in range(len(region)):
            if k in region[i]:
                outpath=root+'train/'+str(i)+'/'+str(i)+'-'+str(k)+'-'+str(j)
                print('outpath',outpath)
        if not os.path.isdir(outpath):
            os.makedirs(outpath)
        out_data=data[j*30000:j*30000+30000]
        out_data = torch.tensor(out_data)
        print('out_data',out_data)
        torch.save(out_data, outpath+'.pt')         
    for j in test_list:
        for i in range(len(region)):
            if k in region[i]:
                outpath=root+'test/'+str(i)+'/'+str(i)+'-'+str(k)+'-'+str(j)
                print('outpath',outpath)
        if not os.path.isdir(outpath):
            os.makedirs(outpath)
        out_data=data[j*30000:j*30000+30000]
        out_data = torch.tensor(out_data)
        print('out_data',out_data)
        torch.save(out_data, outpath+'.pt')

        

In [ ]:
import SailiencyMapping_modules
from SailiencyMapping_modules import  UpstreamExpert
from SailiencyMapping_modules import _get_featurizer
from SailiencyMapping_modules import _get_downstream
from SailiencyMapping_modules import preprocess
import yaml
import torch
import matplotlib.pyplot as plt
from torchsummary import summary
import requests
from PIL import Image
from upstream.mockingjay.builder_refine import PretrainedTransformer_refine
from upstream.mockingjay.builder import PretrainedTransformer

with open('options_arg_config.yaml', 'r') as yml:
    options_arg_config = yaml.safe_load(yml)
    
#-----------extracter
path=options_arg_config['path']
load_weights = torch.load(path)
self_config=load_weights['Upstream_Config']
target_level=25
region=6
#------------PretrainedTransformer


options=options_arg_config['options']
Pretrainedtransformer_refine = PretrainedTransformer_refine(options, inp_dim=-1)

upstreamexpert=UpstreamExpert().eval()
featurizer=_get_featurizer().eval()
downstream=_get_downstream().eval()
#------------------------

pth='$share/ECoG_training/Kin2_20110524_session1_CommonAve_ReginLabel/test/'+str(region)+'/'
#pth='data/'+str(region)+'/'

wavs=[]
label=[]
for i in [3,8,10,11,24,25,34,42]:
    wav = torch.load(pth+str(region)+'-88-'+str(i)+'.pt')
    print('wav',wav)
    wav=torch.tensor(wav, dtype=torch.float32)
    label.append(region)
    wavs.append(wav)
label=torch.tensor(label)
wavs_=[torch.randn(16000) for i in range(8)]
print('wavs_',wavs_)
#wavs = [torch.FloatTensor(wav) for wav in wavs]
x=preprocess(wavs,self_config)
for i in range(len(x)):
    plt.imshow(x[i])
    plt.axis('on')
    plt.show()
#--------------------------
x.requires_grad_()
last_hidden_state,hidden_state=Pretrainedtransformer_refine(x)
feature={
    "last_hidden_state":last_hidden_state,
    "hidden_states":hidden_state.unbind(dim=0),
              }
feature=featurizer(wavs,feature)

scores=downstream(features=feature,labels=label,mode=eval)
print('scores',scores)
for i in range(len(scores)):
    
    score_max_index = scores[i].argmax()
    print('score_max_index',score_max_index )
    score_max = scores[i][score_max_index]
    print('score_max',score_max)
    score_max.backward(retain_graph=True)
    saliency = x.grad.data.abs()
    print('saliency',saliency)
    plt.imshow(saliency[i], cmap=plt.cm.hot)
    plt.axis('off')
    plt.show()
